# Lesson 5 - BOLDdeterministic

The `BOLDdeterministic` module offers a deterministic approach to diffusion modelling. In a complete simulation, it replaces both the `Spins` and `Sequence` objects. To begin we first import the module as follows (and NumPy for array creation, matplotlib for plotting):

In [ ]:
from BOLDswimsuite import BOLDgeometry, BOLDdeterministic
import numpy as np
import matplotlib.pyplot as plt

Deterministic diffusion requires a discrete-space voxel. Rather than using Monte Carlo spins to provide the magnetic field offset samples, we use the points on a discrete grid (from the discrete voxel geometry). Unlike spins however, our samples do not diffuse through space over time, they remain on the grid. Instead, the diffusion is modelled by convolving the resulting magnetization (Mx, My) with a special kernel. In essence we are performing a no-diffusion simulation with the gridded samples, and then we apply a post processing step (kernel convolution) on the magnetization to simulate the effects of diffusion. Currently this method is implemented only for 2D geometry, so we will use that in the examples.

To get started, we once again start by creating a randomly generated 2D continuous voxel.

In [ ]:
random_continuous_voxel = BOLDgeometry.ContinuousVoxel2D.from_random(
    size=0.2,
    CBV=0.02,
    B0=3,
    labels=['vein', 'artery'],
    weights={
        'vein':1, 
        'artery':1
    },
    diameter_distributions={
        'vein': [0.002, 0.003, 0.004], 
        'artery': [0.003, 0.004, 0.005]
    },
    dchis={
        'vein': 3e-8,
        'artery': 4e-8
    },
    permeation_probabilities={
        'vein': 0, 
        'artery': 0
    },
    vessel_type='cylinder',
    allow_vessel_intersection=True,
    seed=0, #repeating with the same seed with provide the same result
    progressbar=True
)

print(f'Number of vessels: {len(random_continuous_voxel.vessels)}')

Since deterministic diffusion requires a discrete voxel, we will now convert the continuous voxel to a discrete voxel using the `from_continuous_analytical` alternate constructor of `DiscreteVoxel2D`.

In [ ]:
discrete_voxel = BOLDgeometry.DiscreteVoxel2D.from_continuous_analytical(
    N=200,
    voxel=random_continuous_voxel
)

We now create a `DeterministicDiffuser2D` object. This object replaces both the `Spins` and `Sequence` objects we were previously using. Many of the arguments we have seen before, and they serve the same purpose (i.e. defining the diffusion length and the pulse sequence). There are two new arguments to define for this object.
- kernel_type : Literal['ModifiedBessel', 'Gaussian'], the type of convolution kernel to use. Default is 'ModifiedBessel'.
- permeable_vessels : bool, if False, will use a correction method to stop diffusion across vessel walls. Otherwise vessels will be permeable. Default is False.

The kernel type can remain unchanged, as the modified Bessel function kernel is more accurate. Unlike our simulations with `Spins`, deterministic diffusion does not take in account the permeation probabilities we input in the geometry. This is because vessel permeability has not yet been implemented on a per-vessel basis. We are also unable to assign a probability, we can only choose permeable or impermeable vessels. This is why the `permeable_vessels` argument is necessary, it will therefore overwrite any previously assigned permeation probabilities.

In [ ]:
dt=0.2 #we will use a constant 0.2ms time step

dd2d = BOLDdeterministic.DeterministicDiffuser2D(
    geometry=discrete_voxel,
    pulse_time_indices=[0, 50],                     #[0ms       , 10ms]
    pulse_angles=[np.pi/2, np.pi],                  #[90 degrees, 180 degrees]
    pulse_axes=[[np.pi/2, np.pi/2], [np.pi/2, 0]],  #[y-axis    , x-axis]
    ADC=0.001,
    dt=dt,
    kernel_type='ModifiedBessel',
    permeable_vessels=False
)

We see that the deterministic diffuser prints out the kernel size. This is important as we need the kernel size to be at least larger than 3 (ideally upwards of 10). If the kernel is too small there will be a warning. The kernel size is increased either by increasing `N` during voxel creation, or by increasing the `ADC` (which is not usually desired).

Much like the `Sequence` object, we can now call `step` or `walk` on our `DeterministicDiffuser2D` to advance the simulation and obtain the signals. Here we will just use the `walk` method for convenience.

In [ ]:
num_steps = 200 # 200*0.2ms = 40ms

eviv, ev, iv = dd2d.walk(
    dt=dt,
    num_steps=num_steps,
    progressbar=True    
)

Now we can plot the signals using matplotlib.

In [ ]:
# array of the time range
time_range = np.arange(0, dt*num_steps, dt)

# creating a matplotlib figure
figure, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

# plotting all three signals with some formatting
ax1.plot(time_range, eviv)
ax1.set_title('Total')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Signal')

ax2.plot(time_range, ev)
ax2.set_title('EV')
ax2.set_xlabel('Time (ms)')

ax3.plot(time_range, iv)
ax3.set_title('IV')
ax3.set_xlabel('Time (ms)')

figure.tight_layout()

This completes the basic lessons for BOLDswimsuite! There are some example scripts of common simulations in the examples folder which can be used as reference:
- `3D-ANA-MC_script.py`: Monte Carlo diffusion simulation of a randomly generated 3D continuous voxel.

- `3D-ANA-MC-G_script.py`: Monte Carlo diffusion simulation of a randomly generated 3D discrete voxel with analytical offset calculation.

- `3D-FFT-MC_script.py`: Monte Carlo diffusion simulation of a randomly generated 3D discrete voxel with FFT offset calculation.

- `2D-ANA-DD_script.py`: deterministic diffusion simulation of a randomly generated 2D discrete voxel.

- `2D-ANA-MC_script.py`: Monte Carlo diffusion simulation of a randomly generated 2D continuous voxel.

- `2D-ANA-MC-G_script.py`: Monte Carlo diffusion simulation of a randomly generated 2D discrete voxel with analytical offset calculation.
